In [1]:
## Importing the required python libraries 
import os, sys
import warnings
import timeit
import numpy as np
import pandas as pd 
import pkg_resources
import matplotlib.pyplot as plt
import re
import seaborn as sns
from collections import defaultdict


In [2]:
## Importing pergene files 

pergene_files=[]
#data_dir= "../satay/data_files/data_unmerged/"
#data_dir="../transposonmapper/data_files/files4test/"
data_dir="../postprocessed-data/"
#data_dir="../transposonmapper/data_files/"
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith("pergene_insertions.xlsx"):
            pergene_files.append(os.path.join(root, file))

list_data=[]
for i in pergene_files:
    list_data.append(pd.read_excel(i,engine='openpyxl',index_col="Unnamed: 0"))

keys=[]
for i in np.arange(0,len(pergene_files)):
    keys.append(pergene_files[i].split("/")[-1].split("_")[0]+"_"+pergene_files[i].split("/")[-1].split("_")[1])

list_data_pd=pd.concat(list_data,axis=0,keys=keys)

In [3]:
## Importing postproccesed data from pipeline

data_post=[]

for i in keys:
    data_post.append(pd.read_excel(data_dir+i+".xlsx",index_col="Unnamed: 0",engine='openpyxl'))
    
list_data_post=pd.concat(data_post,axis=0,keys=keys)
list_data_post.drop(columns=["Feature_alias","Feature_name"],inplace=True)
list_data_post.fillna(1,inplace=True)


In [4]:
list_data_post.head(2)
# list_data_pd.head()

Standard_name Feature_type  Essentiality Chromosome  Position  \
dbem3_b 0     noncoding            1           1.0          I    [0, 1]   
        1        TEL01L     Telomere           1.0          I  [2, 335]   

           Nbasepairs  Ninsertions  Ninsertions_truncatedgene  Nreads  \
dbem3_b 0           2            0                          0       0   
        1         334           38                         38    2356   

                                                 Nreads_list  \
dbem3_b 0                                               [0.]   
        1  [  0.   0.   0.   0.   0.   0.   0.   0.   0. ...   

           Nreads_truncatedgene  Nreadsperinsrt  Nreadsperinsrt_truncatedgene  
dbem3_b 0                     0        0.000000                           0.0  
        1                  2356       63.675676                          62.0

In [5]:
## Importing gene protein length , and protein domains coordinates of every gene 

protein_length=pd.read_csv('../data/Gene_Protein_length_yeastmine.tsv',sep="\t")
protein_length.index=protein_length.loc[:,"Gene Name"]
protein_length.drop(columns="Gene Name",inplace=True)

domains_coordinates=pd.read_csv('../data/Domains_all_genes_protein_coordinates_yeastmine.tsv',sep="\t")

domains_coordinates.dropna(inplace=True)
gene_with_introns=pd.read_csv('../data/Gene_Introns.tsv',sep="\t")



In [6]:
## Combine protein length and protein domains coordinates
domains_coordinates.loc[:,"Protein Length"]=np.nan
for gene_name in protein_length.index: 
    tmp=np.where(domains_coordinates.loc[:,"Gene Name"]==gene_name)[0]
    if tmp.size!=0: 
        for i in np.arange(0,len(tmp)):
            domains_coordinates.loc[tmp[i], "Protein Length"]=protein_length.loc[gene_name, "Protein Length"]

domains_coordinates.dropna(inplace=True) # to drop the proteins that do not have protein domains annotated
domains_coordinates.index=domains_coordinates.loc[:,"Gene Name"]
domains_coordinates.drop(columns="Gene Name",inplace=True)

In [7]:
domains_coordinates.head(2)

,Protein Domain,Protein domain description,Protein start location,Protein End location,Method,Domain ID,Protein Length
Gene Name,,,,,,,
AAC1,PF00153,Mitochondrial carrier protein; Mitochondrial s...,10,106,Pfam,2106639,309.0
AAC1,PF00153,Mitochondrial carrier protein; Mitochondrial s...,116,210,Pfam,2106640,309.0


In [8]:
## Add for every gene name from the pergene files , a column with an array of the protein domains
# founded in the domains coordinates file, as well as the length of the protein,and the
# end location and start location of every protein domain

tmp=np.where(list_data_pd.loc[keys[0],"Gene name"]=="AAC1")[0][0]
list_data_domains_data=list_data_pd.copy()

df=list_data_domains_data.loc[keys[0]]

df.loc[tmp,"Protein Domain"]=np.array(domains_coordinates.loc["AAC1","Protein Domain"])


/data/localhome/linigodelacruz/anaconda3/envs/transposonmapper/lib/python3.9/site-packages/pandas/core/indexing.py:1678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = empty_value


In [15]:
df[df.loc[:,"Gene name"]=="AAC1"]

,Gene name,Chromosome,Start location,End location,Insertion locations,Reads per insertion location,Reads,Insertions,Protein Domain
4756,AAC1,XIII,387315,388244,"[387338, 387376, 387380, 387386, 387393, 38740...","[9, 89, 2, 2, 1, 2, 4, 4, 2, 1, 1, 5, 3, 1, 1,...",2563,77,"[PF00153, PF00153, PF00153]"


In [24]:
domains_coordinates.head()

b=[]

for i in np.arange(0,len(domains_coordinates.loc["AAC1","Protein Domain"])):
    b.append(np.array([domains_coordinates.loc["AAC1","Protein start location"][i],domains_coordinates.loc["AAC1",
"Protein End location"][i]]))

df.loc[:,"Protein domains locations"]=np.zeros_like(df.loc[:,"Protein Domain"])
#df.loc[tmp,"Protein domains locations"]=np.array(b)


/data/localhome/linigodelacruz/anaconda3/envs/transposonmapper/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [25]:
df.loc[tmp,"Protein domains locations"]

0

In [ ]:
df.loc[tmp,"Protein Domain"]

array(['PF00153', 'PF00153', 'PF00153'], dtype=object)

In [ ]:
df.loc[tmp,"Gene name"]

'AAC1'